In [ ]:
import torch
import torch.nn
import numpy as np
import scipy
import scipy.io
import PhysicsInformedROCDF
import matplotlib.pyplot as plt
from matplotlib import cm
# make sure "../Fqoi.mat" can be found

#### Visualization

In [ ]:
data = scipy.io.loadmat("./Fqoi.mat")
data.keys()

In [ ]:
tgrid = data['t_grid']
tmin, tmax = min(tgrid), max(tgrid)
nt = len(tgrid)
print("Range of tgrid = {}, {}, num points = {}".format(tmin, tmax, nt))
xgrid = data['x_grid']
xmin, xmax = min(xgrid), max(xgrid)
nx = len(xgrid)
print("Range of xgrid = {}, {}, num points = {}".format(xmin, xmax, nx))
# mc solution
u_data = data['Fqoi']
print("Shape of u_data = {}".format(u_data.shape))
# visualize solution
xgrid_plot, tgrid_plot = np.meshgrid(xgrid, tgrid)
plt.figure(1, figsize=(10, 8));
fig, ax = plt.subplots(subplot_kw={"projection": "3d"})
surf = ax.plot_surface(xgrid_plot, tgrid_plot, u_data.T, cmap=cm.coolwarm,
                       linewidth=0, antialiased=False);
ax.set_xlabel("space (x)"); ax.set_ylabel("time (t)"); ax.set_zlabel("Solution u(x, t)");

In [ ]:
diffusion = data['diffusion']
print("Shape of diffusion = {}".format(diffusion.shape))
hmean = data['hmean']
print("Shape of <h> = {}".format(hmean.shape))
theta = data['theta']
print("Shape of theta = {}".format(theta.shape))
Dgam = data['Dgam'][0][0]

The stored shape of the Monte Carlo solution $F_{\text{MC}}$ is $N_x \times N_t$, we preprocess the diffusion and advection data with the same shape according to the following formulas. The diffusion is already computed in MATLAB and only needs to be repeated along the $x$-axis.

$$
    \langle v(x,t)\rangle = \frac{1}{D_{\gamma}}(\theta(x) - \langle h(t)\rangle)
$$

In [ ]:
# compute advection at each grid point
advection = (theta - hmean.T)
plt.figure(1, figsize=(10, 8));
fig, ax = plt.subplots(subplot_kw={"projection": "3d"})
surf = ax.plot_surface(xgrid_plot, tgrid_plot, advection.T, cmap=cm.coolwarm,
                       linewidth=0, antialiased=False);
ax.set_xlabel("space (x)"); ax.set_ylabel("time (t)"); ax.set_zlabel("Advection <v(x, t)>");

In [ ]:
# compute diffusion at each grid point
diffusion_plot = np.repeat(diffusion.T, nx, axis=0)
plt.figure(1, figsize=(10, 8));
fig, ax = plt.subplots(subplot_kw={"projection": "3d"})
surf = ax.plot_surface(xgrid_plot, tgrid_plot, diffusion_plot.T, cmap=cm.coolwarm,
                       linewidth=0, antialiased=False);
ax.set_xlabel("space (x)"); ax.set_ylabel("time (t)"); ax.set_zlabel("Diffusion(x, t)");

Save preprocessed data

In [ ]:
scipy.io.savemat("./PINNCDF/Fqoi.mat", {"Dgam": Dgam, "advection": advection, "diffusion": diffusion_plot, 
                                       "x_grid": xgrid, "t_grid": tgrid, "Fqoi": u_data})

### Development Experiments

In [ ]:
pinn = PhysicsInformedROCDF.PhysicsInformedROCDF(2, 1, Fmc_data_path="/Fqoi.mat")

In [ ]:
X = pinn.pde_data[0]
y_true = pinn.fmc_data

In [ ]:
y_true[X[:, 1][:, None] == pinn.xmax]

In [ ]:
X[:, 0]

In [ ]:
pinn.tmin

In [ ]:
xmesh, tmesh = np.meshgrid(pinn.xgrid, pinn.tgrid)

In [ ]:
xmesh

In [ ]:
all_pnts = np.vstack([xmesh.ravel(), tmesh.ravel()]).T

In [ ]:
plt.plot(y_true[all_pnts[:, 0] == pinn.xmin])

In [ ]:
plt.plot(y_true[all_pnts[:, 0] == pinn.xmax])

In [ ]:
plt.plot(y_true[all_pnts[:, 1] == pinn.tmin])